In [ ]:
%pip install neo4j-driver

##### Dataset
The original dataset can be found in "https://github.com/krlawrence/graph"

In [1]:
#import libraries
from neo4j import GraphDatabase, basic_auth
import pandas as pd

In [6]:
#define sandbox credentials
uri= "bolt://3.87.55.10:7687"
pwd = "rains-adaption-figure"

In [7]:
#establish connection to the Graph Database
driver = GraphDatabase.driver(uri, auth=basic_auth("neo4j", pwd))
driver

In [8]:
# Function to execute the query and return the results
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        # Extract the results as a list of dictionaries
        data = [record["label"] for record in result]
    return data

# Execute the query
node_labels = execute_query(driver, query = 'CALL db.labels()')

# Print the results
print("Node Labels:", node_labels)

Node Labels: ['Airport', 'City', 'Region', 'Country', 'Continent']


In [9]:
# Function to execute the query and return the results
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        data = [record["relationshipType"] for record in result]
    return data

relationship_types = execute_query(driver, query = 'CALL db.relationshipTypes()')
# Print the results
print("Relationship Types:", relationship_types)

Relationship Types: ['IN_CITY', 'IN_COUNTRY', 'IN_REGION', 'ON_CONTINENT', 'HAS_ROUTE']


In [10]:
# Initialize an empty list to store the results
data = []

In [11]:
#Query the number of nodes
query = "MATCH (n) RETURN COUNT(n)"

In [12]:
with driver.session() as session:
    result = session.run(query)
    print(result.single())

<Record COUNT(n)=8627>


In [13]:
with driver.session() as session:
    result = session.run(query)
    print(result.single()["COUNT(n)"])

8627


In [14]:
with driver.session() as session:
    result = session.run(query)
    count = result.single()["COUNT(n)"]
    data.append({"Entity": "Nodes", "Count": count})
    print(f'Number of Nodes: {count}')

Number of Nodes: 8627


In [15]:
# Query number of Country nodes
query = "MATCH (n:Country) RETURN COUNT(n)"

In [16]:
with driver.session() as session:
    result = session.run(query)
    # Get the single result record
    print(result.single())

<Record COUNT(n)=232>


In [17]:
with driver.session() as session:
    result = session.run(query)
    # Get the single result record
    print(result.single()["COUNT(n)"])

232


In [18]:
with driver.session() as session:
    result = session.run(query)
    count = result.single()["COUNT(n)"]
    data.append({"Entity": "Countries", "Count": count})
    print(f'Number of countries: {count}')

Number of countries: 232


In [19]:
# Query number of Airports
query = "MATCH (n:Airport) RETURN COUNT(n)"

In [20]:
with driver.session() as session:
    result = session.run(query)
    count = result.single()["COUNT(n)"]
    data.append({"Entity": "Airports", "Count": count})
    print(f'Number of Airports: {count}')

Number of Airports: 3503


In [21]:
# Query number of Cities
query = "MATCH (n:City) RETURN COUNT(n)"

In [22]:
with driver.session() as session:
    result = session.run(query)
    count = result.single()["COUNT(n)"]
    data.append({"Entity": "Cities", "Count": count})
    print(f'Number of Cities: {count}')

Number of Cities: 3359


In [23]:
# Query number of Regions
query = "MATCH (n:Region) RETURN COUNT(n)"

In [24]:
with driver.session() as session:
    result = session.run(query)
    count = result.single()["COUNT(n)"]
    data.append({"Entity": "Regions", "Count": count})
    print(f'Number of Regions: {count}')

Number of Regions: 1527


In [25]:
# Query number of Continents
query = "MATCH (n:Continent) RETURN COUNT(n)"

In [26]:
with driver.session() as session:
    result = session.run(query)
    count = result.single()["COUNT(n)"]
    data.append({"Entity": "Continents", "Count": count})
    print(f'Number of Continents: {count}')

Number of Continents: 6


In [27]:
# Convert the list of results into a pandas DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

       Entity  Count
0       Nodes   8627
1   Countries    232
2    Airports   3503
3      Cities   3359
4     Regions   1527
5  Continents      6


Distribution of the number of airports per continent 

In [28]:
# Define the Cypher query
query = """
MATCH (:Airport)-[:ON_CONTINENT]->(c:Continent)
RETURN c.name AS continentName, count(*) AS numAirports
ORDER BY numAirports DESC
"""

# Function to execute the query and return the results
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        # Extract the results as a list of dictionaries
        data = [{"continentName": record["continentName"], "numAirports": record["numAirports"]} for record in result]
    return data

# Execute the query
results = execute_query(driver, query)
results

[{'continentName': 'NA', 'numAirports': 989},
 {'continentName': 'AS', 'numAirports': 971},
 {'continentName': 'EU', 'numAirports': 605},
 {'continentName': 'AF', 'numAirports': 321},
 {'continentName': 'SA', 'numAirports': 313},
 {'continentName': 'OC', 'numAirports': 304}]

In [29]:
# Convert the results to a DataFrame
airports_df = pd.DataFrame(results)

# Print the DataFrame
print(airports_df)

  continentName  numAirports
0            NA          989
1            AS          971
2            EU          605
3            AF          321
4            SA          313
5            OC          304


Calculate the minimum, maximum, average, and standard deviation of the number of flights out of each airport.

In [30]:
# Define the Cypher query
query = """
MATCH (a:Airport)-[:HAS_ROUTE]->(:Airport)
WITH a, count(*) AS numberOfRoutes
RETURN 
    min(numberOfRoutes) AS minRoutes, 
    max(numberOfRoutes) AS maxRoutes, 
    avg(numberOfRoutes) AS avgRoutes, 
    stdev(numberOfRoutes) AS stdDevRoutes
"""

# Function to execute the query and return the results
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        # Extract the results as a list of dictionaries
        data = [record.data() for record in result]
    return data

# Execute the query
results = execute_query(driver, query)

In [31]:
# Convert the results to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the DataFrame
print(df)

   minRoutes  maxRoutes  avgRoutes  stdDevRoutes
0          1        307  20.905363     38.287309


Create Node Graph Projection

In [32]:
query = """
CALL gds.graph.project(
    'airportGraph',
    'Airport',
    {
        HAS_ROUTE: {
            type: 'HAS_ROUTE',
            orientation: 'UNDIRECTED'
        }
    }
)
YIELD
    graphName,
    nodeProjection,
    nodeCount,
    relationshipProjection,
    relationshipCount,
    projectMillis
"""


In [33]:
def create_graph_projection(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# Execute the query
projection_results = create_graph_projection(driver, query)

# Convert the results to a DataFrame
projection_df = pd.DataFrame(projection_results)
projection_df

,graphName,nodeProjection,nodeCount,relationshipProjection,relationshipCount,projectMillis
0,airportGraph,"{'Airport': {'label': 'Airport', 'properties':...",3503,"{'HAS_ROUTE': {'aggregation': 'DEFAULT', 'orie...",92778,9301


Create Node Graph Projection

In [34]:
# Define the Cypher query for graph projection
query = """
CALL gds.graph.project(
    'routes',
    'Airport',
    'HAS_ROUTE'
)
YIELD
    graphName, nodeProjection, nodeCount, relationshipProjection, relationshipCount
"""

# query = """
# CALL gds.graph.project(
#     'routes-airport',
#     {
#         Airport: {
#             label: 'Airport',
#             properties: ['iata', 'city']
#         }
#     },
#     'HAS_ROUTE'
# )
# YIELD graphName, nodeCount, relationshipCount;
# """


# Function to execute the query and return the results
def create_graph_projection(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# Execute the query
projection_results = create_graph_projection(driver, query)

In [35]:
# Convert the results to a DataFrame
projection_df = pd.DataFrame(projection_results)

# Adjust Pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)        

# Display the DataFrame
projection_df

,graphName,nodeProjection,nodeCount,relationshipProjection,relationshipCount
0,routes,"{'Airport': {'label': 'Airport', 'properties':...",3503,"{'HAS_ROUTE': {'aggregation': 'DEFAULT', 'orie...",46389


In [36]:
node_projection = projection_df['nodeProjection'].iloc[0]
node_projection

{'Airport': {'label': 'Airport', 'properties': {}}}

In [37]:
relationship_projection = projection_df['relationshipProjection'].iloc[0]
relationship_projection

{'HAS_ROUTE': {'aggregation': 'DEFAULT',
  'orientation': 'NATURAL',
  'indexInverse': False,
  'properties': {},
  'type': 'HAS_ROUTE'}}

List All Graphs

In [38]:
# Define the Cypher query for listing GDS graphs
query = """
CALL gds.graph.list()
"""

# Function to execute the query and return the results
def list_gds_graphs(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# Execute the query
graph_list_results = list_gds_graphs(driver, query)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.list' is deprecated.)} {position: line: 2, column: 1, offset: 1} for query: '\nCALL gds.graph.list()\n'


In [39]:
# Convert the results to a DataFrame
graph_list_df = pd.DataFrame(graph_list_results)

pd.set_option('display.max_columns', None)  
pd.set_option('display.width', 1000)        
pd.set_option('display.max_rows', None)      
pd.set_option('display.max_colwidth', None)  

# Display the DataFrame
graph_list_df

,degreeDistribution,graphName,database,databaseLocation,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema,schemaWithOrientation
0,"{'min': 0, 'max': 307, 'p90': 38, 'p999': 282, 'p99': 175, 'p50': 2, 'p75': 9, 'p95': 71, 'mean': 13.242649157864687}",routes,neo4j,local,870 KiB,891472,3503,46389,"{'relationshipProjection': {'HAS_ROUTE': {'aggregation': 'DEFAULT', 'orientation': 'NATURAL', 'indexInverse': False, 'properties': {}, 'type': 'HAS_ROUTE'}}, 'readConcurrency': 4, 'relationshipProperties': {}, 'nodeProperties': {}, 'jobId': '57f358d3-a724-4ecc-b13e-09e41c151a38', 'nodeProjection': {'Airport': {'label': 'Airport', 'properties': {}}}, 'logProgress': True, 'creationTime': 2024-08-04T19:02:19.890901098+00:00, 'validateRelationships': False, 'sudo': False}",0.003781,2024-08-04T19:02:19.890901098+00:00,2024-08-04T19:02:20.090307987+00:00,"{'graphProperties': {}, 'nodes': {'Airport': {}}, 'relationships': {'HAS_ROUTE': {}}}","{'graphProperties': {}, 'nodes': {'Airport': {}}, 'relationships': {'HAS_ROUTE': {'direction': 'DIRECTED', 'properties': {}}}}"
1,"{'min': 0, 'max': 610, 'p90': 72, 'p999': 562, 'p99': 334, 'p50': 4, 'p75': 18, 'p95': 134, 'mean': 26.485298315729374}",airportGraph,neo4j,local,1126 KiB,1153640,3503,92778,"{'relationshipProjection': {'HAS_ROUTE': {'aggregation': 'DEFAULT', 'orientation': 'UNDIRECTED', 'indexInverse': False, 'properties': {}, 'type': 'HAS_ROUTE'}}, 'readConcurrency': 4, 'relationshipProperties': {}, 'nodeProperties': {}, 'jobId': 'bb6bdd2a-6c81-4042-a893-31f28a9c2dcb', 'nodeProjection': {'Airport': {'label': 'Airport', 'properties': {}}}, 'logProgress': True, 'creationTime': 2024-08-04T19:01:57.296289175+00:00, 'validateRelationships': False, 'sudo': False}",0.007563,2024-08-04T19:01:57.296289175+00:00,2024-08-04T19:02:06.674598214+00:00,"{'graphProperties': {}, 'nodes': {'Airport': {}}, 'relationships': {'HAS_ROUTE': {}}}","{'graphProperties': {}, 'nodes': {'Airport': {}}, 'relationships': {'HAS_ROUTE': {'direction': 'UNDIRECTED', 'properties': {}}}}"


# Graph Algorithms

*PageRank Algorithm*

In [40]:
# Define the Cypher query for PageRank
query = """
CALL gds.pageRank.stream('routes')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score AS pageRank
RETURN n.iata AS iata, n.descr AS description, pageRank
ORDER BY pageRank DESC, iata ASC
"""

# Function to execute the query and return the results
def execute_pagerank_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# Execute the query
pagerank_results = execute_pagerank_query(driver, query)

# Convert the results to a DataFrame
pagerank_df = pd.DataFrame(pagerank_results)


# Display the DataFrame
print(pagerank_df.head())

  iata                                         description   pageRank
0  DFW             Dallas/Fort Worth International Airport  11.979783
1  ORD                Chicago O'Hare International Airport  11.162988
2  DEN                        Denver International Airport  10.997299
3  ATL  Hartsfield - Jackson Atlanta International Airport  10.389948
4  IST                      Istanbul International Airport   8.425801


In [41]:
# Define the Cypher query to write Page Rank Algorithm Results to the Database
query = """
CALL gds.pageRank.write('routes',
    {
        writeProperty: 'pageRank'
    }
)
YIELD nodePropertiesWritten, ranIterations
"""

# Function to execute the query and return the results
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# Execute the query
results = execute_query(driver, query)

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Display the DataFrame
print(df)

   nodePropertiesWritten  ranIterations
0                   3503             20


In [42]:
# Define the Cypher query after applying Page Rank Algorithm to the database
query = """
MATCH (a:Airport)
RETURN a.iata AS iata, a.descr AS description, a.pageRank AS pageRank
ORDER BY a.pageRank DESC, a.iata ASC
"""

# Function to execute the query and return the results
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# Execute the query
results = execute_query(driver, query)

# Convert the results to a DataFrame
df = pd.DataFrame(results)

pd.set_option('display.max_columns', None)   
pd.set_option('display.width', 1000)      
pd.set_option('display.max_rows', None)    
pd.set_option('display.max_colwidth', None) 

# Display the DataFrame
print(df.head())

  iata                                         description   pageRank
0  DFW             Dallas/Fort Worth International Airport  11.979783
1  ORD                Chicago O'Hare International Airport  11.162988
2  DEN                        Denver International Airport  10.997299
3  ATL  Hartsfield - Jackson Atlanta International Airport  10.389948
4  IST                      Istanbul International Airport   8.425801


*Community (cluster) detection via Louvain Modularity Algorithm*

In [43]:
# Define the Cypher query to Extract Louvain Algorithm Results 
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

query = """
CALL gds.louvain.stream('routes')
YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) AS airport, communityId
RETURN
    communityId,
    COUNT(airport) AS numberOfAirports,
    COLLECT(DISTINCT airport.city) AS cities,
    COLLECT(DISTINCT airport.iata) AS iataCodes
ORDER BY numberOfAirports DESC, communityId;
"""

try:
    results = execute_query(driver, query)
    print("Louvain Algorithm Results:\n")
except Exception as e:
    print(f"Error running Louvain Modularity Algorithm: {e}")

# Convert the results to a DataFrame
df = pd.DataFrame(results)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
df.head()

Louvain Algorithm Results:



,communityId,numberOfAirports,cities,iataCodes
0,3018,686,"[Atlanta, Anchorage, Austin, Nashville, Boston, Baltimore, Washington D.C., Dallas, Fort Lauderdale, Houston, New York, Los Angeles, Orlando, Miami, Minneapolis, Chicago, West Palm Beach, Phoenix, Raleigh, Seattle, San Francisco, San Jose, Tampa, San Diego, Long Beach, Santa Ana, Salt Lake City, Las Vegas, Denver, White Plains, San Antonio, New Orleans, Newark, Cedar Rapids, Honolulu, El Paso, San Juan, Cleveland, Oakland, Tucson, Santa Fe, Philadelphia, Detroit, Toronto, Vancouver, Ottawa, Fort Myers, Montreal, Edmonton, Calgary, St. John's, Mexico City, Kingston, Tallahassee, Pittsburgh, Portland, Oaklahoma City, Ontario, Rochester, Halifax, Winnipeg, Charlotte, Cancun, Palm Springs, Memphis, Cincinnati, Indianapolis, Kansas City, St Louis, Albuquerque, Milwaukee, Harrison, Salina, Omaha, Tulsa, Puerto Vallarta, Kahului, Nassau, Freeport, George Town, Key West, Bridgetown, St. George, Charlotte Amalie, Hamilton, Scarborough, Port of Spain, Montego Bay, Little Rock, Curacao, Kralendijk, Oranjestad, Norfolk, Jacksonville, Providence, Punta Cana, Harrisburg, Sacramento, Roatan Island, Tegucigalpa, ...]","[ATL, ANC, AUS, BNA, BOS, BWI, DCA, DFW, FLL, IAD, IAH, JFK, LAX, LGA, MCO, MIA, MSP, ORD, PBI, PHX, RDU, SEA, SFO, SJC, TPA, SAN, LGB, SNA, SLC, LAS, DEN, HPN, SAT, MSY, EWR, CID, HNL, HOU, ELP, SJU, CLE, OAK, TUS, SAF, PHL, DTW, YYZ, YVR, YOW, RSW, YUL, YEG, YYC, YYT, MEX, KIN, TLH, PIT, PWM, PDX, OKC, ONT, ROC, RST, YHZ, YWG, CLT, CUN, PSP, MEM, CVG, IND, MCI, DAL, STL, ABQ, MKE, MDW, HRO, SLN, OMA, TUL, PVR, OGG, NAS, FPO, GGT, EYW, BGI, ANU, STT, BDA, TAB, POS, MBJ, LIT, CUR, BON, AUA, ORF, ...]"
1,3307,522,"[London, Paris, Frankfurt, Helsinki, Dublin, Rome, Amsterdam, Prague, Barcelona, Madrid, Vienna, Zurich, Geneva, Brussels, Munich, Manchester, Cologne, Gothenburg, Venice, Shannon, Oslo, Stockholm, Nottingham, Edinburgh, Glasgow, Liverpool, Nice, Milan, Athens, Zagreb, Budapest, Alicante, Bilbao, Ibiza, Menorca, Tenerife, Larnaca, Warsaw, Luqa, Sofia, Belgrade, Tel Aviv, Hamburg, Stuttgart, Genoa, Naples, Pisa, Turin, Bologna, Verona, Nantes, Copenhagen, Luxembourg, Dusseldorf, Lisbon, Gibraltar, Tunis, Reykjavik, Gran Canaria, Southampton, Palma De Mallorca, Riga, Malaga, Funchal, Leeds, Aberdeen, Antalya, Saint Helier, Zakynthos, Rhodes, Bristol, Newcastle, Saint Peter Port, Eindhoven, Sevilla, Basle, Dubrovnik, Stavanger, Bergen, Tallinn, Algiers, Cork, Wroclaw, Split, Belfast, Hannover, Lyon, Marseille, Bucharest, Rotterdam, Casablanca, Tangier, Faro, Mykonos Island, Santorini Island, Kiev, Rijeka, Toulouse/Blagnac, Porto, Culleredo, ...]","[LHR, LGW, CDG, FRA, HEL, DUB, FCO, AMS, PRG, BCN, MAD, VIE, ZRH, GVA, BRU, MUC, MAN, CGN, LCY, GOT, VCE, SNN, OSL, ARN, STN, EMA, EDI, GLA, LPL, ORY, NCE, MXP, ATH, ZAG, BUD, ALC, BIO, IBZ, MAH, TFN, LCA, WAW, MLA, SOF, BEG, TLV, HAM, STR, GOA, NAP, PSA, TRN, BLQ, TSF, VRN, NTE, CPH, LUX, DUS, LIS, GIB, TUN, LTN, KEF, LPA, SOU, PMI, RIX, AGP, FNC, LBA, ABZ, AYT, JER, ZTH, RHO, BRS, NCL, GCI, EIN, SVQ, BSL, DBV, SVG, BGO, TLL, ALG, ORK, WRO, SPU, BHD, HAJ, LIN, LYS, MRS, OTP, RTM, CMN, TNG, FAO, ...]"
2,2784,377,"[Dubai, New Delhi, Mumbai, Doha, Calicut, Hyderabad, Chennai, Kolkata, Bengaluru, Cape Town, Johannesburg, Durban, Nairobi, Mombasa, Cairo, Addis Ababa, Kuwait, Istanbul, Manama, Abu Dhabi, Colombo, Kathmandu, Jeddah, Muscat, Lagos, Harare, Luxor, Riyadh, Islamabad, Amman, Karachi, Lahore, Ashgabat, Jaipur, Accra, Kampala, Abuja, Beirut, Freetown, Luanda, Tripoli, Dhaka, Sylhet, Tehran, Port Louis, Mahe Island, Malé, Kigali, Arusha, Ad Dammam, Arbil, Dakar, Agra, Khajuraho, Varanasi, Mangalore, Ahmedabad, Jodhpur, Pune, Sharjah, Aden, Coimbatore, Cochin, Trivandrum, Tiruchirappally, Sana'a, Diu, Porbandar, Windhoek, Ankara, Lusaka, Hargeisa, Berbera, Djibouti City, Alexandria, Port Sudan, Juba, Khartoum, Dar es Salaam, Gheshm, Kabul, Douala, Cotonou, Ouagadougou, Abidjan, Niamey, Sfax, Lomé, Brazzaville, Pointe 

*Node similarity Algorithm*

In [44]:
# Define the Cypher query to Extract Node similarity Algorithm Results 
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

query = """
CALL gds.nodeSimilarity.stream('routes')
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity
RETURN
    n1.iata AS iata,
    n1.city AS city,
    COLLECT({iata:n2.iata, city:n2.city, similarityScore: similarity}) AS similarAirports
ORDER BY city LIMIT 20
"""

try:
    results = execute_query(driver, query)
    print("Node similarity Algorithm Results:\n")
except Exception as e:
    print(f"Error running Node similarity Algorithm: {e}")

# Convert the results to a DataFrame
df = pd.DataFrame(results)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
df.head()

Node similarity Algorithm Results:



,iata,city,similarAirports
0,AAL,Aalborg,"[{'iata': 'KRS', 'city': 'Kjevik', 'similarityScore': 0.3333333333333333}, {'iata': 'HAU', 'city': 'Karmøy', 'similarityScore': 0.2727272727272727}, {'iata': 'SZZ', 'city': 'Goleniow', 'similarityScore': 0.2608695652173913}, {'iata': 'AAR', 'city': 'Aarhus', 'similarityScore': 0.25}, {'iata': 'TRF', 'city': 'Torp', 'similarityScore': 0.24444444444444444}, {'iata': 'BLL', 'city': 'Billund', 'similarityScore': 0.23333333333333334}, {'iata': 'AES', 'city': 'Ålesund', 'similarityScore': 0.22727272727272727}, {'iata': 'GOT', 'city': 'Gothenburg', 'similarityScore': 0.21875}, {'iata': 'SVG', 'city': 'Stavanger', 'similarityScore': 0.21568627450980393}, {'iata': 'KYA', 'city': 'Konya', 'similarityScore': 0.21052631578947367}]"
1,AAR,Aarhus,"[{'iata': 'AAL', 'city': 'Aalborg', 'similarityScore': 0.25}, {'iata': 'GOT', 'city': 'Gothenburg', 'similarityScore': 0.234375}, {'iata': 'TLL', 'city': 'Tallinn', 'similarityScore': 0.22916666666666666}, {'iata': 'VXO', 'city': 'Växjö', 'similarityScore': 0.21739130434782608}, {'iata': 'SVG', 'city': 'Stavanger', 'similarityScore': 0.21153846153846154}, {'iata': 'HAU', 'city': 'Karmøy', 'similarityScore': 0.20833333333333334}, {'iata': 'KRS', 'city': 'Kjevik', 'similarityScore': 0.20833333333333334}, {'iata': 'OSD', 'city': 'Östersund', 'similarityScore': 0.2}, {'iata': 'AOI', 'city': 'Ancona', 'similarityScore': 0.19230769230769232}, {'iata': 'XRY', 'city': 'Jerez de la Forntera', 'similarityScore': 0.1875}]"
2,YXX,Abbotsford,"[{'iata': 'YQU', 'city': 'Grande Prairie', 'similarityScore': 0.2857142857142857}, {'iata': 'YQR', 'city': 'Regina', 'similarityScore': 0.23529411764705882}, {'iata': 'YQQ', 'city': 'Comox', 'similarityScore': 0.2222222222222222}, {'iata': 'YHM', 'city': 'Hamilton', 'similarityScore': 0.21052631578947367}, {'iata': 'YXJ', 'city': 'Fort St.John', 'similarityScore': 0.2}, {'iata': 'YXE', 'city': 'Saskatoon', 'similarityScore': 0.2}, {'iata': 'YLW', 'city': 'Kelowna', 'similarityScore': 0.1875}, {'iata': 'YXU', 'city': 'London', 'similarityScore': 0.17647058823529413}, {'iata': 'YMM', 'city': 'Fort McMurray', 'similarityScore': 0.16666666666666666}, {'iata': 'YXS', 'city': 'Prince George', 'similarityScore': 0.14285714285714285}]"
3,ABR,Aberdeen,"[{'iata': 'HIB', 'city': 'Hibbing', 'similarityScore': 1.0}, {'iata': 'BJI', 'city': 'Bemidji', 'similarityScore': 1.0}, {'iata': 'ISN', 'city': 'Williston', 'similarityScore': 0.5}, {'iata': 'ATY', 'city': 'Watertown', 'similarityScore': 0.5}, {'iata': 'RHI', 'city': 'Rhinelander', 'similarityScore': 0.5}, {'iata': 'INL', 'city': 'International Falls', 'similarityScore': 0.5}, {'iata': 'APN', 'city': 'Alpena', 'similarityScore': 0.5}, {'iata': 'BRD', 'city': 'Brainerd', 'similarityScore': 0.5}, {'iata': 'DIK', 'city': 'Dickinson', 'similarityScore': 0.5}, {'iata': 'LSE', 'city': 'La Crosse', 'similarityScore': 0.3333333333333333}]"
4,ABZ,Aberdeen,"[{'iata': 'INV', 'city': 'Inverness', 'similarityScore': 0.3023255813953488}, {'iata': 'MJV', 'city': 'San Javier', 'similarityScore': 0.2708333333333333}, {'iata': 'JER', 'city': 'Saint Helier', 'similarityScore': 0.25862068965517243}, {'iata': 'GLA', 'city': 'Glasgow', 'similarityScore': 0.24528301886792453}, {'iata': 'NCL', 'city': 'Newcastle', 'similarityScore': 0.24050632911392406}, {'iata': 'BHD', 'city': 'Belfast', 'similarityScore': 0.23529411764705882}, {'iata': 'GOT', 'city': 'Gothenburg', 'similarityScore': 0.23170731707317074}, {'iata': 'BLL', 'city': 'Billund', 'similarityScore': 0.22784810126582278}, {'iata': 'ORK', 'city': 'Cork', 'similarityScore': 0.22727272727272727}, {'iata': 'EDI', 'city': 'Edinburgh', 'similarityScore': 0.22377622377622378}]"


In [45]:
# Define the Cypher query to Extract Node similarity Algorithm Results for  top 3 most similar airports for each airport node in the graph
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

query = """
CALL gds.nodeSimilarity.stream(
    'routes',
    {
        topK: 3
    }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity
RETURN
    n1.iata AS iata,
    n1.city AS city,
    COLLECT({iata:n2.iata, city:n2.city, similarityScore: similarity}) AS similarAirports
ORDER BY city LIMIT 20
"""

try:
    results = execute_query(driver, query)
    print("Node similarity Algorithm Results:\n")
except Exception as e:
    print(f"Error running Node similarity Algorithm: {e}")

# Convert the results to a DataFrame
df = pd.DataFrame(results)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
df

Node similarity Algorithm Results:



,iata,city,similarAirports
0,AAL,Aalborg,"[{'iata': 'KRS', 'city': 'Kjevik', 'similarityScore': 0.3333333333333333}, {'iata': 'HAU', 'city': 'Karmøy', 'similarityScore': 0.2727272727272727}, {'iata': 'SZZ', 'city': 'Goleniow', 'similarityScore': 0.2608695652173913}]"
1,AAR,Aarhus,"[{'iata': 'AAL', 'city': 'Aalborg', 'similarityScore': 0.25}, {'iata': 'GOT', 'city': 'Gothenburg', 'similarityScore': 0.234375}, {'iata': 'TLL', 'city': 'Tallinn', 'similarityScore': 0.22916666666666666}]"
2,YXX,Abbotsford,"[{'iata': 'YQU', 'city': 'Grande Prairie', 'similarityScore': 0.2857142857142857}, {'iata': 'YQR', 'city': 'Regina', 'similarityScore': 0.23529411764705882}, {'iata': 'YQQ', 'city': 'Comox', 'similarityScore': 0.2222222222222222}]"
3,ABR,Aberdeen,"[{'iata': 'HIB', 'city': 'Hibbing', 'similarityScore': 1.0}, {'iata': 'BJI', 'city': 'Bemidji', 'similarityScore': 1.0}, {'iata': 'ISN', 'city': 'Williston', 'similarityScore': 0.5}]"
4,ABZ,Aberdeen,"[{'iata': 'INV', 'city': 'Inverness', 'similarityScore': 0.3023255813953488}, {'iata': 'MJV', 'city': 'San Javier', 'similarityScore': 0.2708333333333333}, {'iata': 'JER', 'city': 'Saint Helier', 'similarityScore': 0.25862068965517243}]"
5,AHB,Abha,"[{'iata': 'ELQ', 'city': 'Buraidah', 'similarityScore': 0.5}, {'iata': 'YNB', 'city': 'Yenbo', 'similarityScore': 0.4375}, {'iata': 'HAS', 'city': 'Ha'il', 'similarityScore': 0.4}]"
6,ABJ,Abidjan,"[{'iata': 'BKO', 'city': 'Senou', 'similarityScore': 0.5}, {'iata': 'OUA', 'city': 'Ouagadougou', 'similarityScore': 0.4444444444444444}, {'iata': 'DKR', 'city': 'Dakar', 'similarityScore': 0.4}]"
7,ABI,Abilene,"[{'iata': 'ACT', 'city': 'Waco', 'similarityScore': 1.0}, {'iata': 'BPT', 'city': 'Beaumont/Port Arthur', 'similarityScore': 1.0}, {'iata': 'TXK', 'city': 'Texarkana', 'similarityScore': 1.0}]"
8,AZI,Abu Dhabi,"[{'iata': 'XSB', 'city': 'Sir Bani Yas Island', 'similarityScore': 0.2}, {'iata': 'BDH', 'city': 'Bandar Lengeh', 'similarityScore': 0.2}, {'iata': 'LFM', 'city': 'Lamerd', 'similarityScore': 0.16666666666666666}]"
9,AUH,Abu Dhabi,"[{'iata': 'DOH', 'city': 'Doha', 'similarityScore': 0.43781094527363185}, {'iata': 'MCT', 'city': 'Muscat', 'similarityScore': 0.4032258064516129}, {'iata': 'DXB', 'city': 'Dubai', 'similarityScore': 0.37254901960784315}]"


In [46]:
# Define the Cypher query to Extract Node similarity Algorithm Results for each airport and then returned the 10 airport pairs with the highest similarity across the whole graph.
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

query = """
CALL gds.nodeSimilarity.stream(
    'routes',
    {
        topK: 1,
        topN: 10
    }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity AS similarityScore
RETURN
    n1.iata AS iata,
    n1.city AS city,
    {iata:n2.iata, city:n2.city} AS similarAirport,
    similarityScore
ORDER BY city
"""

try:
    results = execute_query(driver, query)
    print("Node similarity Algorithm Results:\n")
except Exception as e:
    print(f"Error running Node similarity Algorithm: {e}")

# Convert the results to a DataFrame
df = pd.DataFrame(results)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
df

Node similarity Algorithm Results:



,iata,city,similarAirport,similarityScore
0,ABI,Abilene,"{'iata': 'TXK', 'city': 'Texarkana'}",1.0
1,AEX,Alexandria,"{'iata': 'GRK', 'city': 'Fort Hood/Killeen'}",1.0
2,BPT,Beaumont/Port Arthur,"{'iata': 'TXK', 'city': 'Texarkana'}",1.0
3,CLL,College Station,"{'iata': 'LCH', 'city': 'Lake Charles'}",1.0
4,DRO,Durango,"{'iata': 'SAF', 'city': 'Santa Fe'}",1.0
5,GCK,Garden City,"{'iata': 'TXK', 'city': 'Texarkana'}",1.0
6,IAG,Niagara Falls,"{'iata': 'PSM', 'city': 'Portsmouth'}",1.0
7,SAF,Santa Fe,"{'iata': 'DRO', 'city': 'Durango'}",1.0
8,TXK,Texarkana,"{'iata': 'ABI', 'city': 'Abilene'}",1.0
9,ACT,Waco,"{'iata': 'TXK', 'city': 'Texarkana'}",1.0


In [47]:
# Define the Cypher query to Extract Node similarity Algorithm Results for each airport and then returned the 10 airport pairs with the lowest similarity across the whole graph.
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

query = """
CALL gds.nodeSimilarity.stream(
    'routes',
    {
        topK: 1,
        bottomN: 10
    }
)
YIELD node1, node2, similarity
WITH gds.util.asNode(node1) AS n1, gds.util.asNode(node2) AS n2, similarity AS similarityScore
RETURN
    n1.iata AS iata,
    n1.city AS city,
    {iata:n2.iata, city:n2.city} AS similarAirport,
    similarityScore
ORDER BY city
"""

try:
    results = execute_query(driver, query)
    print("Node similarity Algorithm Results:\n")
except Exception as e:
    print(f"Error running Node similarity Algorithm: {e}")

# Convert the results to a DataFrame
df = pd.DataFrame(results)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
df

Node similarity Algorithm Results:



,iata,city,similarAirport,similarityScore
0,VLY,Angelsey,"{'iata': 'GCI', 'city': 'Saint Peter Port'}",0.045455
1,BET,Bethel,"{'iata': 'ANI', 'city': 'Aniak'}",0.071429
2,BNB,Boende,"{'iata': 'FIH', 'city': 'Kinshasa'}",0.058824
3,HMV,Hemavan,"{'iata': 'ARN', 'city': 'Stockholm'}",0.005952
4,HJR,Khajuraho,"{'iata': 'JAI', 'city': 'Jaipur'}",0.058824
5,MMU,Morristown,"{'iata': 'MSN', 'city': 'Madison'}",0.052632
6,PRI,Praslin Island,"{'iata': 'DUR', 'city': 'Durban'}",0.062500
7,ISC,St. Mary's,"{'iata': 'EGC', 'city': 'Bergerac/Roumanière'}",0.071429
8,TVF,Thief River Falls,"{'iata': 'AZA', 'city': 'Phoenix'}",0.017857
9,WRL,Worland,"{'iata': 'DEN', 'city': 'Denver'}",0.004630
